## Import modules

In [1]:
import sys
sys.path.append('src')

In [2]:
import pandas
import numpy

from energiapy.components import *
from energiapy.utils.data_utils import make_henry_price_df, remove_outliers

## Data Import

The following data is needed for the model

- solar and wind profiles : energiapy.fetch_nsrdb_data imports data from the NREL NSRDB database
- power demand : ERCOT for Houston; CAISO for San Diego
- Natural Gas prices: Henry Hub Price Index for both


**Get Weather data**

In [3]:
def load_data(loc:str, index:list):
    df = pandas.read_csv(f'data/{loc}_solar19.csv')
    df['idx'] = index
    df = df.set_index('idx')
    return df

In [4]:
idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]


In [5]:
weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]


**Demand data for San Diego (CAISO for SDGE region) and Houston (ERCOT for COAST region)**

In [6]:
demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
demand_ny = pandas.DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga

**Natural gas prices from Henry Hub Price Index**  

We use the special function energiapy.make_henry_price_df because of special requirements such as filling in empty data points (weekends, public holidays). Such function will be generalized in the next update of energipy

In [7]:
ng_price = make_henry_price_df(
    file_name='Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
ng_price = ng_price.drop(columns= 'scales')
ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
ng_price = pandas.concat([ng_price])

## Define temporal scale


The variabilities of energy systems are best captured over a discretized spatio-temporal scale. In energiapy, the first declaration is the temporal scale. 

For e.g.: Here we declare three temporal scales at different levels from right to left. The interpretation of these scales is merely symentic. Scales can be declared as the problem demands.
- 0, annual, with 1 discretization
- 1, daily with 365 discretization
- 2, hourly with 24 discretization

In essence, we are creating a temporal scale of 8760 points.

In [8]:
scales = TemporalScale(discretization_list=[1, 365, 24], design_scale= 0, scheduling_scale=2)

In [9]:
bigM = 10**6

## Declare resources

Resources can be consumed, produced, stored, discharged (or sold). 

In [10]:
th_h2 = Theta((0, 10))

In [11]:
# Hydrogen = Resource(name='H2', discharge = True, sell_price = Theta((0, 10)))

In [12]:
Solar = Resource(name='Solar', cons_max=100, basis='MW', label='Solar Power')

Wind = Resource(name='Wind', cons_max= 100, basis='MW', label='Wind Power')

Power = Resource(name='Power', basis='MW', discharge= True, label='Power generated')

Uranium = Resource(name='Ur', cons_max=100, purchase_price= 42.70/(250/2), basis='kg', label='Uranium')

H2 = Resource(name='H2', basis='tons', label='Hydrogen')

H2_L = Resource(name='H2_L', store_max = 10000, basis='tons', label='Hydrogen')

CO2_AQoff = Resource(name='CO2(Aq)', store_max = 10000, basis='tons', label='Carbon dioxide - sequestered')

H2O = Resource(name='H2O', cons_max=100,
               purchase_price= 0.001, basis='tons', label='Water')

CH4 = Resource(name='CH4', cons_max=100, purchase_price=1, basis='tons', label='Natural gas')

CO2 = Resource(name='CO2', basis='tons', label='Carbon dioxide', block='Resource')

CO2_Vent = Resource(name='CO2_Vent', discharge= True, basis='tons', label='Carbon dioxide - Vented')

O2 = Resource(name='O2', discharge=True, basis='tons', label='Oxygen')

CO2_DAC = Resource(name='CO2_DAC', basis='tons', label='Carbon dioxide - captured')


In [13]:
ProcessType.uncertain()


['VOPEX',
 'CAPACITY',
 'CAPEX',
 'FOPEX',
 'CREDIT',
 'LAND',
 'CAP_MAX',
 'INCIDENTAL']

In [15]:
ResourceType.uncertain()

['SELL_PRICE',
 'AVAILABILITY',
 'STORE_LOSS',
 'STORE_MAX',
 'PURCHASE_PRICE',
 'STORAGE_COST',
 'DEMAND']

In [21]:
from enum import Enum, auto

In [26]:
class C(Enum):
    pass 


for i in ProcessType.parameters():
    print(i)
    setattr(C, i, auto())    

CAP_MAX
CAP_MIN
CAPEX
FOPEX
VOPEX
INCIDENTAL
INTRODUCE
RETIRE
LIFETIME
TRL
P_FAIL
LAND
CREDIT


In [20]:
LocationType.parameters()

['LAND_COST', 'LAND_MAX']

In [15]:
ResourceType.resource_level_classifications()

['PURCHASE',
 'PRODUCE',
 'CONSUME',
 'MEET_DEMAND',
 'STORE',
 'SELL',
 'IMPLICIT',
 'DISCHARGE']

## Declare processes

In [16]:
LiI = Process(name='LiI', storage= Power, capex = 3516428, fopex= 87910, vopex = 0, store_max = 10000, cap_max=1000, label='Lithium-ion battery', basis = 'MW')

# WF = Process(name='WF', conversion={Wind: -1, Power: 1}, capex= (0, 20), fopex= (0, 3), vopex=(0, 1), cap_max= Theta((1, 2)), land = Theta((4, 10)), label='Wind mill array', basis = 'MW')

WF = Process(name='WF', conversion={Wind: -1, Power: 1},capex=1462000, fopex=43000, vopex=4953, cap_max=1000, land = 30, label='Wind mill array', basis = 'MW')

PV = Process(name='PV', conversion={Solar: -1, Power: 1}, capex = 1333262, fopex= 22623, cap_max=1000, label = 'Solar PV', basis = 'MW')

SMRH = Process(name='SMRH', conversion={Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 1.03, CO2: 9.332}, capex =2520000, fopex = 945000, vopex = 51.5, cap_max= 1000, label='Steam methane reforming + CCUS')

NGCC = Process(name= 'NGCC', conversion = {Power: 1, CH4: -0.108, CO2_Vent: 0.297*0.05, CO2:0.297*0.95}, capex = 2158928, fopex= 53320, vopex = 4090, cap_max= 1000, label = 'NGCC + 95% CC')

SMR = Process(name='SMR', capex = 2400, fopex = 800, vopex = 0.03,  conversion={Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 9.4979}, cap_max=1000, label='Steam methane reforming')

H2FC = Process(name='H2FC', conversion = {H2:-0.050, Power: 1}, capex =  1.6*10**6, vopex = 3.5, cap_max = 1000, label = 'hydrogen fuel cell')

DAC = Process(name='DAC', capex = 0.02536, fopex = 0.634, conversion={Power: -0.193, H2O: -4.048, CO2_DAC: 1}, cap_max=1000, label='Direct air capture')

PSH = Process(name='PSH', storage= Power, capex = 3924781, fopex= 17820, vopex = 512.5, store_max = 10000, cap_max=1000, label='Pumped storage hydropower', basis = 'MW')

H2_L_c = Process(name='H2_L_c', conversion={Power: -0.417, H2_L: 1, H2: -1}, capex =  1.6*10**6, vopex = 3.5, cap_max= 1000, label='Hydrogen geological storage')

H2_L_d = Process(name='H2_L_d', conversion={H2_L: -1, H2: 1}, capex =  0.01, vopex = 0.001, cap_max= 1000, label='Hydrogen geological storage discharge')

DAC = Process(name='DAC', conversion={Power: -0.193, H2O: -4.048, CO2_DAC: 1}, capex = 730, fopex= 114, vopex= 3.6, cap_max=1000, label='Direct air capture')

ASMR = Process(name='ASMR', conversion={Uranium: -4.17*10**(-5), H2O: -3.364, Power: 1}, capex = 7988951, fopex= 0.04*0.730, cap_max=1000, label='Small modular reactors (SMRs)')

AWE = Process(name='AWE', land = 20,  conversion={Power: -1, H2: 0.019, O2: 0.7632, H2O: -0.1753}, capex = 1.1*10**6, fopex = 16918, cap_max=1000, label='Alkaline water electrolysis (AWE)', citation='Demirhan et al. 2018 AIChE paper') 

AQoff_SMR = Process(name='AQoff_SMR', conversion={Power: -1.28, CO2_AQoff: 1, CO2: -1}, capex =  5.52, vopex = 4.14, cap_max= 1000,  label='Offshore aquifer CO2 sequestration (SMR)')

AttributeError: process_level_classification

In [16]:
f'{ProcessType}'

"<enum 'ProcessType'>"

In [16]:
PV.ptype

{<ProcessType.CAP_MAX: 12>: <ParameterType.CERTAIN: 1>,
 <ProcessType.CAPEX: 14>: <ParameterType.CERTAIN: 1>,
 <ProcessType.FOPEX: 15>: <ParameterType.CERTAIN: 1>,
 <ProcessType.CAPACITY: 11>: <ParameterType.CERTAIN: 1>,
 <ProcessType.LINEAR_CAPEX: 9>: <ParameterType.CERTAIN: 1>,
 <ProcessType.SINGLE_PRODMODE: 1>: <ParameterType.CERTAIN: 1>,
 <ProcessType.NO_MATMODE: 3>: <ParameterType.CERTAIN: 1>}

In [18]:
set(Process.parameters_declared_here()) & set(Process.mpvar_parameters())

{'cap_max', 'cap_min', 'capex', 'fopex', 'incidental', 'land', 'vopex'}

In [17]:
MPVarType.process()

['CAPACITY', 'CAPEX', 'FOPEX', 'VOPEX', 'INCIDENTAL', 'CREDIT']

In [ ]:
ProcessType.process_level()

['SINGLE_PRODMODE',
 'MULTI_PRODMODE',
 'NO_MATMODE',
 'SINGLE_MATMODE',
 'MULTI_MATMODE',
 'STORAGE',
 'STORAGE_DISCHARGE',
 'STORAGE_REQ',
 'LINEAR_CAPEX',
 'PWL_CAPEX',
 'CAPACITY',
 'CAP_MAX',
 'CAP_MIN',
 'CAPEX',
 'FOPEX',
 'VOPEX',
 'INCIDENTAL',
 'INCIDENTAL',
 'INTRODUCE',
 'RETIRE',
 'LIFETIME',
 'TRL',
 'P_FAIL',
 'LAND']

In [15]:
ProcessType._member_names_

['SINGLE_PRODMODE',
 'MULTI_PRODMODE',
 'NO_MATMODE',
 'SINGLE_MATMODE',
 'MULTI_MATMODE',
 'STORAGE',
 'STORAGE_DISCHARGE',
 'STORAGE_REQ',
 'LINEAR_CAPEX',
 'PWL_CAPEX',
 'CAPACITY',
 'CAP_MAX',
 'CAP_MIN',
 'CAPEX',
 'FOPEX',
 'VOPEX',
 'INCIDENTAL',
 'INTRODUCE',
 'RETIRE',
 'LIFETIME',
 'TRL',
 'P_FAIL',
 'LAND',
 'CREDIT']

In [15]:
process_list = {LiI, WF, PV, SMRH, SMR, DAC, AWE, H2_L_c, H2_L_d, AQoff_SMR, ASMR, PSH, NGCC}

In [17]:
PV.ptype

{<ProcessType.CAPACITY: 11>: <ParameterType.CERTAIN: 1>,
 <ProcessType.CAP_MAX: 12>: <ParameterType.CERTAIN: 1>,
 <ProcessType.CAPEX: 14>: <ParameterType.CERTAIN: 1>,
 <ProcessType.LINEAR_CAPEX: 9>: <ParameterType.CERTAIN: 1>,
 <ProcessType.FOPEX: 15>: <ParameterType.CERTAIN: 1>,
 <ProcessType.SINGLE_PRODMODE: 1>: <ParameterType.CERTAIN: 1>,
 <ProcessType.NO_MATMODE: 3>: <ParameterType.CERTAIN: 1>}

## Declare locations

In [18]:
cc = Factor(data = pandas.DataFrame({'a': [2]}), scales= scales, apply_max_scaler= False)

houston = Location(name='HO', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, credit = {PV: 2}, cap_max_localize = {WF: 0.4, AWE: 23}, 
                   credit_factor ={PV: pandas.DataFrame({'a': [2]})}, cons_max_localize = {Solar: 2}, demand= {Power: (0, 1)}, 
                   land_cost = 10, land_cost_factor= cc, capacity_factor={PV: weather_ho[['dni']], WF: weather_ho[['wind_speed']]}, demand_factor={
                   Power: demand_ho}, purchase_price_factor={CH4: ng_price}, scales=scales, label='Houston')

sandiego = Location(name='SD', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, purchase_price_localize = {CH4: 222}, 
                    cons_max_localize = {Solar: 3}, land_cost = 30, land_cost_factor = pandas.DataFrame({'a': [2]}) , 
                    credit = {SMRH: 30, AQoff_SMR: Theta((0, 23))}, credit_factor= {SMRH: pandas.DataFrame({'a': [1 - 0.001*i for i in range(365)]})}, 
                    capacity_factor={PV: weather_sd[['dni']], WF: weather_sd[['wind_speed']]}, demand_factor={
                    Power: demand_sd}, purchase_price_factor={CH4: ng_price},  scales=scales, label='SanDiego')

newyork = Location(name='NY', processes={LiI, H2FC}, land_max = (0, 50000), land_max_factor = pandas.DataFrame({'a': [2]}), cap_max_localize = {PV: 0.2, AWE: 34}, 
                   capacity_factor={PV: weather_ny[['dni']], WF: weather_ny[['wind_speed']]}, demand= {Power: 40}, demand_factor={
                   Power: demand_ny}, purchase_price_localize = {CH4: 2}, scales=scales, label='NewYork')


In [19]:
houston.resources_demand

{Power}

## Declare transports

In [20]:

# Train_H2 = Transport(name='Train_H2', resources={
#                      H2}, trans_max=bigM, trans_loss=0.03, trans_cost=1.667*10**(-3), label='Railway for hydrogen transportation')
# Grid = Transport(name='Grid', resources={
#                  Power}, trans_max=bigM, trans_loss=0.001, trans_cost=0.5*10**(-3), label='Railroad transport')


Train_H2 = Transport(name='Train_H2', resources={
                     H2}, cap_max=bigM, trans_loss=0.03, capex = 1000, vopex=1.667, fopex = (0,1), label='Railway for hydrogen transportation')
Grid = Transport(name='Grid', resources={
                 Power}, cap_max=bigM, trans_loss=0.001, capex = 500, vopex = 0.444, label='Railroad transport')



## Declare Network

In [21]:
distance_matrix = [
    [2366], 
    [2620]
]

transport_matrix = [
    [[Train_H2]], 
    [[Train_H2]],
]
capacity_factor  = {(houston, sandiego): {Train_H2: weather_sd[['wind_speed']]}}

network = Network(name='Network', scales = scales, land_max= (0,5), land_max_factor= pandas.DataFrame({'a': [5]}), capacity_factor= capacity_factor, sources=[houston, sandiego], sinks=[newyork], distance_matrix=distance_matrix, transport_matrix=transport_matrix)


## Declare scenario

In [29]:
Process.__dict__

mappingproxy({'__module__': 'energiapy.components.process',
              '__annotations__': {'name': str,
               'cap_max': typing.Union[float, typing.Tuple[float], energiapy.components.parameters.mpvar.Theta],
               'cap_min': typing.Union[float, dict],
               'land': typing.Union[float, typing.Tuple[float], energiapy.components.parameters.mpvar.Theta],
               'conversion': typing.Union[typing.Dict[typing.Union[int, str], typing.Dict[energiapy.components.resource.Resource, float]], typing.Dict[energiapy.components.resource.Resource, float]],
               'material_cons': typing.Union[typing.Dict[typing.Union[int, str], typing.Dict[energiapy.components.material.Material, float]], typing.Dict[energiapy.components.material.Material, float]],
               'capex': typing.Union[float, dict, typing.Tuple[float], energiapy.components.parameters.mpvar.Theta],
               'fopex': typing.Union[float, typing.Tuple[float], energiapy.components.parameters.

In [22]:

scenario = Scenario(name='scenario_full', network=network, scales=scales, label='full_case')


In [27]:
Process.__dict__

mappingproxy({'__module__': 'energiapy.components.process',
              '__annotations__': {'name': str,
               'cap_max': typing.Union[float, typing.Tuple[float], energiapy.components.parameters.mpvar.Theta],
               'cap_min': typing.Union[float, dict],
               'land': typing.Union[float, typing.Tuple[float], energiapy.components.parameters.mpvar.Theta],
               'conversion': typing.Union[typing.Dict[typing.Union[int, str], typing.Dict[energiapy.components.resource.Resource, float]], typing.Dict[energiapy.components.resource.Resource, float]],
               'material_cons': typing.Union[typing.Dict[typing.Union[int, str], typing.Dict[energiapy.components.material.Material, float]], typing.Dict[energiapy.components.material.Material, float]],
               'capex': typing.Union[float, dict, typing.Tuple[float], energiapy.components.parameters.mpvar.Theta],
               'fopex': typing.Union[float, typing.Tuple[float], energiapy.components.parameters.

In [47]:
scenario.consider_vopex

True

In [ ]:
ProcessType.__dict__

In [24]:
houston.resources_produce

{CO2, CO2(Aq), CO2_Vent, H2, LiI_Power_stored, O2, Power}

In [ ]:
houston.ptype

In [ ]:
network.transport_avail_dict

In [ ]:
houston.resources_demand

In [ ]:
houston.resources_demand

In [ ]:
houston.demand

In [ ]:
{i: i.name for i in {}}

In [ ]:
if houston.resources_sell:
    print('adsad')

In [ ]:
houston.availability_factor

In [ ]:
network.capacity_factor

In [ ]:
scenario.capacity_factor

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= houston, process= PV, fig_size= (9,5), color= 'orange')


![cap fac pv](plots/cf_ho.png)

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= sandiego, process= WF, fig_size= (9,5), color= 'blue')


![cap fac wf](plots/wf_sd.png)

In [ ]:
plot_scenario.demand_factor(scenario = scenario, location= newyork, resource= Power, fig_size= (9,5), color= 'red')


![df ny](plots/df_ny.png)

## Optimize for minimum cost

In [ ]:
milp_cost = formulate(scenario= scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.COST)

**Ensure no discharge of power in Houston and San Diego**

While the demand for power is zero in these locations, Power is still a dischargeable resource. Hence, the discharge bounds need to be set to zero.
If not set to zero, the system chooses to dispense power instead of establishing storage networks.

In [ ]:

milp_cost.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_cost.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_cost = solve(scenario = scenario, instance= milp_cost, solver= 'gurobi', name="results_cost", print_solversteps = True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_cost, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny](plots/sch_sdny.png)

In [ ]:
plot_results.transport(results=results_cost, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch hony](plots/sch_hony.png)

**Cost contribution by technology**

In [ ]:
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.CAPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.VOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.FOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.TOTAL, location= 'HO', fig_size= (8,6))

![total cost ho](plots/total_ho.png)

**Location-wise cost contribution by technology**

In [ ]:

plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.CAPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.VOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.FOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.TOTAL, fig_size= (8,6))

![total lw](plots/total_lw.png)

**Inventory and production schedules**

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'Inv', component= 'LiI_Power_stored', location = 'SD', fig_size= (9,5), color = 'steelblue')

![inv lii](plots/lii_inv.png)

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'P', component= 'PV', location = 'SD', fig_size= (9,5), color = 'steelblue')

![pv p](plots/pv_p.png)

## Optimize to maximize resource discharge 

In [ ]:

milp_demand = formulate(scenario=scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                        Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.MAX_DISCHARGE, objective_resource=Power)


In [ ]:

milp_demand.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_demand.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_demand = solve(scenario=scenario, instance=milp_demand, solver='gurobi',
                       name="results_demand", print_solversteps=True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_demand, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny2](plots/sch_sdny2.png)

In [ ]:
plot_results.transport(results=results_demand, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch t hony](plots/sch_hony2.png)